In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [1]:
import os
os.chdir('D:/Capstone/')

In [2]:

# !pip install "tensorflow>=1.7.0"
# !pip install tensorflow-hub

In [3]:
## Import Libraries here
from pathlib import Path
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from numpy import array
import pandas as pd
import string
import os
import glob
from time import time
from keras import models
from keras.applications import densenet
import shutil
import re
import tensorflow as tf
import tensorflow_hub as hub
import pickle



Using TensorFlow backend.


In [4]:
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

In [5]:
# images_zip_path = '/content/drive/My Drive/AIML/Capstone/NLMCXR_png.tgz'
images_unzip_path = 'D:/Capstone/NLMCXR_png/'
# ##extract_all(images_zip_path,images_unzip_path)
# shutil.unpack_archive(images_zip_path, images_unzip_path)

# reports_zip_path = '/content/drive/My Drive/AIML/Capstone/NLMCXR_reports.tgz'
reports_unzip_path = 'D:/Capstone/NLMCXR_reports/'
# ##extract_all(images_zip_path,images_unzip_path)
# shutil.unpack_archive(reports_zip_path, reports_unzip_path)

In [6]:
## Variables
directory_in_str=reports_unzip_path+'/ecgen-radiology'
pathlist = Path(directory_in_str).glob('**/*.xml')
glove_dir = '/content/drive/My Drive/AIML/Capstone/Glove/'
# Below path contains all the images
images = images_unzip_path
train_images = images_unzip_path
proj_path = 'D:/Capstone/'
chexnet_weights = '/content/drive/My Drive/AIML/Capstone/brucechou1983_CheXNet_Keras_0.3.0_weights.h5'

## Pre-Processing

In [7]:
## Extract required XML attributes into a Dataframe here

result=[]
for path in pathlist:
     
     path_in_str = str(path)
     with open(path_in_str, "r") as f:
    
        contents = f.read()

        soup = BeautifulSoup(contents, 'lxml')
        tags = soup.findAll(['abstracttext','parentimage',{'id':True}])
        for t in tags:
          label=t.get("label")
          if label=='FINDINGS':
            f_text=t.text
            
          if label=='IMPRESSION':
            i_text=t.text
           
          for j in t.find_all('figureid'):
            if j.text=='F1':
              id=t["id"]

          
        result.append([path_in_str[43:100],id,f_text,i_text])

                


In [21]:
df=pd.DataFrame(result,columns=["XML_File_Name","Image_File_name","Findings","Impressions"])

In [22]:
df.head()

,XML_File_Name,Image_File_name,Findings,Impressions
0,1.xml,CXR1_1_IM-0001-3001,The cardiac silhouette and mediastinum size ar...,Normal chest x-XXXX.
1,10.xml,CXR10_IM-0002-1001,The cardiomediastinal silhouette is within nor...,No acute cardiopulmonary process.
2,100.xml,CXR100_IM-0002-1001,Both lungs are clear and expanded. Heart and m...,No active disease.
3,1000.xml,CXR1000_IM-0003-1001,There is XXXX increased opacity within the rig...,1. Increased opacity in the right upper lobe w...
4,1001.xml,CXR1001_IM-0004-1001,Interstitial markings are diffusely prominent ...,Diffuse fibrosis. No visible focal acute disease.


In [23]:
df.shape

(3955, 4)

In [24]:
# Findings have blank values (530 rows)
df[df["Findings"]==''].count()

XML_File_Name      530
Image_File_name    530
Findings           530
Impressions        530
dtype: int64

In [25]:
# Update Findings --> Fill blank values with Impressions
df["Findings"]=np.where(df["Findings"]=='',df["Impressions"],df["Findings"])

In [26]:
df[df["Findings"]==''].count()

XML_File_Name      28
Image_File_name    28
Findings           28
Impressions        28
dtype: int64

In [27]:
## Remove Blank reports -- 28 Reports
df_final=df[df["Findings"]!='']

In [28]:
df_final.shape

(3927, 4)

In [29]:
s_findings=dict(zip(df_final.Image_File_name,df_final.Findings))
w_findings=dict(zip(df_final.Image_File_name,df_final.Findings))

In [30]:
list(s_findings.keys())[:5]

['CXR1_1_IM-0001-3001',
 'CXR10_IM-0002-1001',
 'CXR100_IM-0002-1001',
 'CXR1000_IM-0003-1001',
 'CXR1001_IM-0004-1001']

In [31]:
s_findings['CXR1_1_IM-0001-3001']

'The cardiac silhouette and mediastinum size are within normal limits. There is no pulmonary edema. There is no focal consolidation. There are no XXXX of a pleural effusion. There is no evidence of pneumothorax.'

In [32]:
## Clean up sentences
remove = string.punctuation
remove = remove.replace(".", "")
def s_clean_up(anything):
	# prepare translation table for removing punctuation
	table = str.maketrans('', '', remove)
	for key, desc_list in anything.items():

		desc = desc_list
		
		# convert to lower case
		desc = desc.lower()
		# remove punctuation from each token
		desc = desc.translate(table)
		desc = desc.replace('0', 'zero ')
		desc = desc.replace('1', 'one ')
		desc = desc.replace('2', 'two ')
		desc = desc.replace('3', 'three ')
		desc = desc.replace('4', 'four ')
		desc = desc.replace('5', 'five ')
		desc = desc.replace('6', 'six ')
		desc = desc.replace('7', 'seven ')
		desc = desc.replace('8', 'eight ')
		desc = desc.replace('9', 'nine ')
##		desc = re.sub('\d', 'XX', desc)
		# store as string
		anything[key] =  desc
  
# clean findings
s_clean_up(s_findings)
# clean impressions
#clean_up(impressions)

In [33]:
s_findings['CXR1_1_IM-0001-3001']

'the cardiac silhouette and mediastinum size are within normal limits. there is no pulmonary edema. there is no focal consolidation. there are no xxxx of a pleural effusion. there is no evidence of pneumothorax.'

In [39]:
## Clean up words
def w_clean_up(anything):
	# prepare translation table for removing punctuation
	table = str.maketrans('', '', string.punctuation)
	for key, desc_list in anything.items():

		desc = desc_list
		# tokenize
		desc = desc.split(' ')

		desc = [x.replace('0', 'zero ') for x in desc]
		desc = [x.replace('1', 'one ') for x in desc]
		desc = [x.replace('2', 'two ') for x in desc]
		desc = [x.replace('3', 'three ') for x in desc]
		desc = [x.replace('4', 'four ') for x in desc]
		desc = [x.replace('5', 'five ') for x in desc]
		desc = [x.replace('6', 'six ') for x in desc]
		desc = [x.replace('7', 'seven ') for x in desc]
		desc = [x.replace('8', 'eight ') for x in desc]
		desc = [x.replace('9', 'nine ') for x in desc]
        
		# convert to lower case
		desc = [word.lower() for word in desc]
		# remove punctuation from each token
		desc = [w.translate(table) for w in desc]
        

        
		# remove hanging 's' and 'a'
		desc = [word for word in desc if len(word)>1]
		# remove tokens with numbers in them
		desc = [word for word in desc if word.replace(' ','').isalpha()]
    # remove tokens with xxxxx in them
# 		desc = [word.replace('xxxxx','') for word in desc]

		# store as string
		anything[key] =  ' '.join(desc)
  
# clean findings
w_clean_up(w_findings)


In [40]:
w_findings['CXR1_1_IM-0001-3001']

'the cardiac silhouette and mediastinum size are within normal limits there is no pulmonary edema there is no focal consolidation there are no xxxx of pleural effusion there is no evidence of pneumothorax'

In [41]:
# convert the loaded descriptions into a vocabulary of sentences
def s_to_vocabulary(anything):
	# build a list of all description strings
	all_desc = set()
	for key in anything.keys():
		[all_desc.update(anything[key].split('. ')) ]
	return all_desc

# summarize findings vocabulary
s_findings_vocabulary = s_to_vocabulary(s_findings)
print('Original Findings Sentences Vocabulary Size: %d' % len(s_findings_vocabulary))

# summarize vocabulary
#impressions_vocabulary = to_vocabulary(impressions)
#print('Original Impressions Vocabulary Size: %d' % len(impressions_vocabulary))

Original Findings Sentences Vocabulary Size: 6740


In [42]:
a=list(s_findings_vocabulary)

In [49]:
# module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/3"
# embed = hub.Module(module_url)

In [50]:
# def process_text(sentences_list):
#     path = proj_path#'./processed_data'
#     embeddings_file = path+"embeddings-{}.pickle".format(len(sentences_list))
#     if not os.path.isfile(os.path.join(path, embeddings_file)):
#         module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/3"
#         embed = hub.Module(module_url)
#         with tf.Session() as sess:
#             sess.run([tf.global_variables_initializer(), tf.tables_initializer()])
#             sentences_list = sess.run(embed(sentences_list))
#         sentences_list = np.array(sentences_list)
#         sentences_list = np.array([np.reshape(embedding, (len(embedding), 1)) for embedding in sentences_list])
#         pickle.dump(sentences_list, open(embeddings_file, 'wb'))
#     else:
#         sentences_list = pickle.load(open(join(path, embeddings_file), 'rb'))
#     return sentences_list

In [41]:
# path = './processed_data'

In [42]:
# embeddings_file = "embeddings-{}.pickle".format(len(s_findings_vocabulary))

In [43]:
# os.j

In [43]:
# em=process_text(a)

In [44]:
# np.save(proj_path+'embeddings', em, allow_pickle=True, fix_imports=True)

In [45]:
# convert the loaded descriptions into a vocabulary of words
def w_to_vocabulary(anything):
	# build a list of all description strings
	all_desc = set()
	for key in anything.keys():
		[all_desc.update(anything[key].split(' ')) ]
	return all_desc

# summarize findings vocabulary
w_findings_vocabulary = w_to_vocabulary(w_findings)
print('Original Findings Word Vocabulary Size: %d' % len(w_findings_vocabulary))

# summarize vocabulary
#impressions_vocabulary = to_vocabulary(impressions)
#print('Original Impressions Vocabulary Size: %d' % len(impressions_vocabulary))

Original Findings Word Vocabulary Size: 1852


In [46]:
w_findings_vocabulary

{'',
 'paraspinal',
 'severely',
 'lefttoright',
 'xxxxgranulomas',
 'intravenous',
 'fat',
 'pads',
 'knee',
 'obliteration',
 'prostheses',
 'interlobar',
 'nasogastric',
 'very',
 'osteophytic',
 'medially',
 'pneumonitis',
 'destruction',
 'along',
 'reflex',
 'above',
 'cassettes',
 'sagittal',
 'first',
 'consistent',
 'exam',
 'atherosclerosis',
 'attenuation',
 'immediately',
 'consolidations',
 'repositioning',
 'concerning',
 'tortuosity',
 'direct',
 'amputation',
 'recent',
 'fragments',
 'margin',
 'dextroconvex',
 'subglottic',
 'tissue',
 'blunted',
 'overexpanded',
 'picc',
 'axilla',
 'artifact',
 'ectasiatortuosity',
 'leads',
 'scoliotic',
 'suggested',
 'displacement',
 'primary',
 'lymph',
 'alveolar',
 'found',
 'include',
 'margination',
 'tenting',
 'embolism',
 'thickeningscarring',
 'multilevel',
 'pleuralparenchymal',
 'three',
 'fifth',
 'underpenetration',
 'coiled',
 'ventricle',
 'popliteal',
 'hyperlucency',
 'greater',
 'loculation',
 'based',
 'zones',

In [47]:
# save descriptions to file, one per line
def save_anything(anything, filename):
	lines = list()
	for key, desc_list in anything.items():
		lines.append(key + ' ' + desc_list)
	data = '\n'.join(lines)
	file = open(filename, 'w')
	file.write(data)
	file.close()

save_anything(s_findings, 's_findings.txt')
save_anything(w_findings, 'w_findings.txt')

In [48]:
train = (list(train_images+df_final.Image_File_name+'.png'))
print('Dataset: %d' % len(train))

Dataset: 3927


In [49]:
train[1]

'D:/Capstone/NLMCXR_png/CXR10_IM-0002-1001.png'

In [50]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# filename = "findings.txt"
# # load descriptions
# doc = load_doc(filename)
# print(doc[:300])

In [51]:
doc = load_doc('s_findings.txt')
descriptions = dict()

In [52]:
# load clean descriptions as sentences into memory
def load_clean_descriptions(filename, dataset):
  # load document
  doc = load_doc(filename)
  descriptions = dict()
  w_descriptions = dict()
  for line in doc.split('\n'):
    # split line by white space
    tokens = line.split()
    # Split the key out
    t = line.replace(' ','. ',1).split('. ') 
    # split id from description
    image_id, image_desc = tokens[0], t[1:]
    # skip images not in the set
    if train_images+image_id+'.png' in train:
      # create list
      if train_images+image_id+'.png' not in descriptions:
        descriptions[image_id] = list()
        descriptions[image_id].append('<p>')
        w_descriptions[image_id] = list()
      # wrap description in tokens
      for row,i in enumerate(image_desc):
        desc = '<s> ' + ''.join(i) + ' </s>'
        if row == 0:
          desc = '<s> ' + ''.join(i) + ' </s>'
          w_descriptions[image_id].append(desc)
        # store
        descriptions[image_id].append(desc)
    descriptions[image_id].append('</p>')
        
  return descriptions, w_descriptions


# findings
train_s_findings, train_w_findings = load_clean_descriptions('s_findings.txt', train)
print('Findings Descriptions: train=%d' % len(train_s_findings))


Findings Descriptions: train=3668


In [53]:
train_w_findings['CXR2831_IM-1249-1001']

['<s> persistent  eight mm pneumothorax with small amount of pleural effusion </s>']

In [54]:
 train_s_findings['CXR2831_IM-1249-1001']

['<p>',
 '<s> persistent  eight mm pneumothorax with small amount of pleural effusion </s>',
 '<s> heart size normal </s>',
 '<s> enteric tube tip in the stomach </s>',
 '<s> midline abdominal clips </s>',
 '<s> xxxx in the right upper quadrant </s>',
 '<s> bullet overlies l three . </s>',
 '</p>']

In [55]:
# def pickle_op(typ,anything):
#     path = proj_path#'./processed_data'
#     pickle_file = path+typ+".pickle"
#     pickle.dump(anything, open(pickle_file, 'wb'))

    
# # train_s_findings
# # w_findings_vocabulary
# # train_w_findings

In [56]:
# pickle_op('para',train_s_findings)
# pickle_op('first_sent',train_w_findings)
# pickle_op('word_vocab',w_findings_vocabulary)


In [35]:

# Create a list of all image names in the directory
img = glob.glob(images + '*.png')

In [50]:
#train_impression_descriptions.get('CXR1_1_IM-0001-3001.png')

In [51]:
# Create a list of all the training captions for Findings
all_train_s_findings_captions = []
for key, val in train_s_findings.items():
    for cap in val:
        all_train_s_findings_captions.append(cap)
print(len(all_train_s_findings_captions))

all_train_w_findings_captions = []
for key, val in train_w_findings.items():
    for cap in val:
        all_train_w_findings_captions.append(cap)
print(len(all_train_w_findings_captions))

24288
3668


In [52]:
# Consider only words which occur at least 10 times in the corpus
word_count_threshold = 1
findings_word_counts = {}
impressions_word_counts = {}
nsents = 0
for sent in all_train_s_findings_captions:
    nsents += 1
    for w in sent.split(' '):
        findings_word_counts[w] = findings_word_counts.get(w, 0) + 1

findings_w_vocab = [w for w in findings_word_counts if findings_word_counts[w] >= word_count_threshold]
print('preprocessed findings words %d -> %d' % (len(findings_word_counts), len(findings_w_vocab)))

# nsents = 0
# for sent in all_train_impressions_captions:
#     nsents += 1
#     for w in sent.split(' '):
#         impressions_word_counts[w] = impressions_word_counts.get(w, 0) + 1

# impressions_vocab = [w for w in impressions_word_counts if impressions_word_counts[w] >= word_count_threshold]
# print('preprocessed impressions words %d -> %d' % (len(impressions_word_counts), len(impressions_vocab)))

preprocessed findings words 2145 -> 2145


In [53]:
ixtoword = {}
wordtoix = {}

# iixtoword = {}
# iwordtoix = {}

ix = 1
for w in findings_w_vocab:
    wordtoix[w] = ix
    ixtoword[ix] = w
    ix += 1

# ix = 1
# for w in impressions_vocab:
#     iwordtoix[w] = ix
#     iixtoword[ix] = w
#     ix += 1

In [54]:
findings_vocab_size = len(ixtoword) + 1 # one for appended 0's
print(findings_vocab_size)

# impressions_vocab_size = len(iixtoword) + 1 # one for appended 0's
# print(impressions_vocab_size)

2146


In [55]:
# convert a dictionary of clean descriptions to a list of descriptions
def to_lines(descriptions):
	all_desc = list()
	for key in descriptions.keys():
		[all_desc.append(d) for d in descriptions[key]]
	return all_desc

# calculate the length of the description with the most words
def max_length(descriptions):
	lines = to_lines(descriptions)
	return max(len(d.split()) for d in lines)

# determine the maximum sequence length
findings_max_length = max_length(train_s_findings)
print('Findings Description Length: %d' % findings_max_length)

# impressions_max_length = max_length(train_impression_descriptions)
# print('Impressions Description Length: %d' % impressions_max_length)

Findings Description Length: 50


In [322]:
# Load Glove vectors
glove_dir = glove_dir
embeddings_index = {} # empty dictionary
f = open(os.path.join(glove_dir, 'glove.6B.200d.txt'), encoding="utf-8")

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [0]:
embedding_dim = 200

# Get 200-dim dense vector for each of the 10000 words in out vocabulary
embedding_matrix = np.zeros((findings_vocab_size, embedding_dim))

for word, i in wordtoix.items():
    #if i < max_words:
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in the embedding index will be all zeros
        embedding_matrix[i] = embedding_vector

In [324]:
embedding_matrix.shape

(858, 200)

## Pre-Trained DenseNet + CheXNet

In [0]:
model=densenet.DenseNet121(include_top=True, 
                           weights=chexnet_weights, 
                           input_tensor=None, 
                           input_shape=None, 
                           pooling=None, 
                           classes=14)

In [0]:
model.summary()

Model: "densenet121"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
________________________________________________________________________________________